Topic
0. open
1. review
    1.1.1 2이차 내용에 대한 review (naver 영화 한줄평을 이용)
2. topic
    2.1 정수 인코딩 (Integer encoding)
    2.2 원-핫 인코딩 (One-hot encoding)
    2.3 Byte Pair Encoding (BPE)
    2.4 데이터 분리 (Splitting Data)
3. Q & A
4. close

# 1. Review

# Naver 영화 한줄평 crawling & Test preprocessing

In [1]:
import requests
from bs4 import BeautifulSoup

# 한줄평 crawling

In [2]:
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321'

In [4]:
resp = requests.get(url)
html = BeautifulSoup(resp.content, 'lxml')

In [5]:
score_result = html.find('div', {'class' : 'score_result'})
lis = score_result.findAll('li')

In [20]:
comments = []
for li in lis:
    comments.append(li.find('p').text.lstrip().rstrip())

### crawling 결과 파일 저장 / 읽기

In [21]:
with open('comment_1917.txt', 'w', encoding='utf-8') as f:
    for comment in comments :
        f.write(comment + '\n')

In [22]:
movie_comments = []
with open('comment_1917.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        movie_comments.append(line.rstrip('\n'))
print(movie_comments)

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


### 파이썬 리스트 내용을 하나의 문자열로 변경

In [23]:
movie_comment = ''
for comment in movie_comments:
    movie_comment += comment
print(movie_comment)

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.충무로: 이거 어케하는거냐?아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다.


# Text preprocessing

In [24]:
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
import re

In [25]:
okt = Okt()

## 형태소 추출

In [26]:
morphemes = []
for morphs in okt.morphs(movie_comment):
    morphemes.append(morphs)
print(morphemes)

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '.', '충무로', ':', '이', '거', '어케', '하는거냐', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '.']


## 형태소 추출 및 어간 추출

In [27]:
morphemes = []
for morphs in okt.morphs(movie_comment, stem = True): # stem : 어간추출여부
    morphemes.append(morphs)
print(morphemes)

['이', '영화', '는', '미치다', '.', '넷플릭스', '가', '일상', '화', '되다', '시대', '에', '극장', '이', '존재', '하다', '이유', '를', '증명', '해주다', '.', '충무로', ':', '이', '거', '어케', '하다', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받다', '이유', '가', '고스', '란', '히', '녹다', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '하다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣다', '마스터피스', '오스카', '작품', '상', '타다', '하다', '없다', '같다', '.']


## 형태소 추출 및 정규화

In [28]:
morphemes = []
for morphs in okt.morphs(movie_comment, norm = True):
    morphemes.append(morphs)
print(morphemes)

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '가', '일상', '화', '된', '시대', '에', '극장', '이', '존재', '해야하는', '이유', '를', '증명', '해준다', '.', '충무로', ':', '이', '거', '어떻게', '하는거냐', '?', '아카데미', '에서', '촬영상', ',', '음향효과', '상', ',', '시', '각', '효과', '상', '을', '받은', '이유', '가', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '로', '관람', '하', '는걸', '추천', '한다', '.', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '.']


## 명사 분리

In [29]:
morphemes = []
for morphs in okt.nouns(movie_comment):
    morphemes.append(morphs)
print(morphemes)

['이', '영화', '넷플릭스', '일상', '시대', '극장', '존재', '이유', '증명', '충무로', '거', '어케', '아카데미', '촬영상', '음향효과', '효과', '이유', '고스', '란', '영화', '로', '관람', '는걸', '추천', '촬영감독', '영혼', '마스터피스', '오스카', '작품']


## 어절 분리

In [30]:
morphemes = []
for morphs in okt.phrases(movie_comment):
    morphemes.append(morphs)
print(morphemes)

['이 영화', '넷플릭스', '일상화', '일상화된 시대', '극장', '존재', '존재해야하는 이유', '증명', '충무로', '이거', '이거 어케', '아카데미', '촬영상', '음향효과상', '시각효과상', '이유', '고스란', '영화', 'IMAX로', 'IMAX로 관람하는걸', 'IMAX로 관람하는걸 추천', '촬영감독', '촬영감독의 영혼', '마스터피스오스카', '마스터피스오스카 작품상', '시대', '어케', '고스', 'IMAX', '관람하', '는걸', '추천', '영혼', '마스터피스', '오스카', '작품상']


## 품사 태깅

In [31]:
morphemes = []
for morphs in okt.pos(movie_comment, join=True):
    morphemes.append(morphs)
print(morphemes)

['이/Noun', '영화/Noun', '는/Josa', '미쳤다/Adjective', './Punctuation', '넷플릭스/Noun', '가/Josa', '일상/Noun', '화/Suffix', '된/Verb', '시대/Noun', '에/Josa', '극장/Noun', '이/Josa', '존재/Noun', '해야하는/Verb', '이유/Noun', '를/Josa', '증명/Noun', '해준다/Verb', './Punctuation', '충무로/Noun', ':/Punctuation', '이/Determiner', '거/Noun', '어케/Noun', '하는거냐/Verb', '?/Punctuation', '아카데미/Noun', '에서/Josa', '촬영상/Noun', ',/Punctuation', '음향효과/Noun', '상/Suffix', ',/Punctuation', '시/Modifier', '각/Modifier', '효과/Noun', '상/Suffix', '을/Josa', '받은/Verb', '이유/Noun', '가/Josa', '고스/Noun', '란/Noun', '히/Adverb', '녹아있는/Verb', '영화/Noun', './Punctuation', 'IMAX/Alpha', '로/Noun', '관람/Noun', '하/Suffix', '는걸/Noun', '추천/Noun', '한다/Verb', './Punctuation', '촬영감독/Noun', '의/Josa', '영혼/Noun', '까지/Josa', '갈아/Adverb', '넣은/Verb', '마스터피스/Noun', '오스카/Noun', '작품/Noun', '상/Suffix', '탔어도/Verb', '할말/Verb', '없었을것/Adjective', '같다/Adjective', './Punctuation']


## 불용어 파일 Load

In [32]:
stop_words = []
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
for line in lines:
    stop_words.append(line.rstrip('\n'))
    
print(stop_words)

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에도', '대해 말하자면', '뿐이다', '다음에', '반대로', '반대로 말하자면', '이와 반대로',

## 불용어 제거

In [34]:
results = []
for word in okt.morphs(movie_comment):
    if word not in stop_words:
        results.append(word)
        
print(results)

['이', '영화', '는', '미쳤다', '.', '넷플릭스', '일상', '화', '된', '시대', '극장', '이', '존재', '해야하는', '이유', '증명', '해준다', '.', '충무로', ':', '이', '거', '어케', '하는거냐', '?', '아카데미', '촬영상', ',', '음향효과', '상', ',', '시', '효과', '상', '받은', '이유', '고스', '란', '히', '녹아있는', '영화', '.', 'IMAX', '관람', '는걸', '추천', '한다', '.', '촬영감독', '영혼', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '.']


### cleaning (특수문자 제거)

In [36]:
r = '[-=+.#/\?:^$.@*\"~&%.,!\'\|\(\)\[\]\<\>]'

movie_comment = re.sub(r, '', movie_comment)
print(movie_comment)

이 영화는 미쳤다 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다충무로 이거 어케하는거냐아카데미에서 촬영상 음향효과상 시각효과상을 받은 이유가 고스란히 녹아있는 영화 IMAX로 관람하는걸 추천한다촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다


In [37]:
results = []
for word in okt.morphs(movie_comment):
    if word not in stop_words:
        results.append(word)
        
print(results)

['이', '영화', '는', '미쳤다', '넷플릭스', '일상', '화', '된', '시대', '극장', '이', '존재', '해야하는', '이유', '증명', '해준다', '충무로', '이', '거', '어케', '하는거냐', '아카데미', '촬영상', '음향효과', '상', '시', '효과', '상', '받은', '이유', '고스', '란', '히', '녹아있는', '영화', 'IMAX', '관람', '는걸', '추천', '한다', '촬영감독', '영혼', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것']


### NLTK - word_tokenize() 사용 결과

In [38]:
word_tokens = word_tokenize(movie_comment)
print(word_tokens)

['이', '영화는', '미쳤다', '넷플릭스가', '일상화된', '시대에', '극장이', '존재해야하는', '이유를', '증명해준다충무로', '이거', '어케하는거냐아카데미에서', '촬영상', '음향효과상', '시각효과상을', '받은', '이유가', '고스란히', '녹아있는', '영화', 'IMAX로', '관람하는걸', '추천한다촬영감독의', '영혼까지', '갈아넣은', '마스터피스오스카', '작품상', '탔어도', '할말', '없었을것', '같다']


In [40]:
results = []
for word in word_tokens:
    if word not in stop_words:
        results.append(word)
        
print(results)

['이', '영화는', '미쳤다', '넷플릭스가', '일상화된', '시대에', '극장이', '존재해야하는', '이유를', '증명해준다충무로', '이거', '어케하는거냐아카데미에서', '촬영상', '음향효과상', '시각효과상을', '받은', '이유가', '고스란히', '녹아있는', '영화', 'IMAX로', '관람하는걸', '추천한다촬영감독의', '영혼까지', '갈아넣은', '마스터피스오스카', '작품상', '탔어도', '할말', '없었을것']


In [41]:
from konlpy.corpus import kolaw
from konlpy.corpus import kobill

In [42]:
kolaw.open('constitution.txt').read()[:30]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국'

In [43]:
kobill.open('1809890.txt').read()[:30]

'지방공무원법 일부개정법률안\n\n(정의화의원 대표발의 )\n'

## 한나눔 (Hannanum) 형태소 분석기

In [44]:
from konlpy.tag import Hannanum

In [45]:
hannanum = Hannanum()

In [46]:
hannanum.nouns('나는 자연어 처리를 공부한다.')

['나', '자연어', '처리', '공부']

In [47]:
hannanum.pos('나는 자연어 처리를 공부한다.')

[('나', 'N'),
 ('는', 'J'),
 ('자연어', 'N'),
 ('처리', 'N'),
 ('를', 'J'),
 ('공부', 'N'),
 ('하', 'X'),
 ('ㄴ다', 'E'),
 ('.', 'S')]

# 3.1 정수 인코딩 (Integer Encoding)

컴퓨터는 텍스트보다는 숫자를 더 잘 처리할 수 있다. 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법이다. 이러한 기법을 적용하는 첫 단계로 각 단어를 고유한 정수에 Encoding 시키는 전처리 작업이 필요할 때가 있다.

### 1. 정수 인코딩 (Integer Encoding)

단어에 정수를 부여하는 방법 중 하나로 단어를 빈도수 순으로 정렬한 단어 집합(vocabularay)로 만들고, 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여하는 방법이 있다.

### 1) dictionary 사용한 방법

In [48]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [49]:
text = "A barber is a person. a barber is good person. a barber is huge person. he knew A secret! The Secret He Kept is huge secret. Huge secret. His Barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [50]:
# 문장 토큰화
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he knew A secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His Barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [53]:
# 정제와 단어 코튼화
vocab = {}
sentences = []
stop_words = set(stopwords.words('english'))
for i in text:
    sentence = word_tokenize(i) # 단어 토큰화
    result = []
    for word in sentence:
        word = word.lower()
        if word not in stop_words:
            if len(word) > 2:
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] += 1
    
    sentences.append(result)
    
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


### 정수 인코딩을 위한 빈도수 계산

In [55]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [56]:
print(vocab['word'])

2


### 빈도수가 높은 순으로 정렬

In [57]:
vocab_sorted = sorted(vocab.items(), key = lambda x : x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


### 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여

In [59]:
word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted:
    if frequency > 1:
        i += 1
        word_to_index[word] = i
        
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [61]:
vocab_size = 5
words_frequency = [ w for w, c in word_to_index.items() if c >= vocab_size + 1]
for w in words_frequency:
    del word_to_index[w]
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### Out-Of-Vocabulary (OOV, 단어 집합에 없는 단어

In [62]:
word_to_index['OOV'] = len(word_to_index) + 1

In [64]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


### word_to_index의 내용을 활용하여 sentences의 모든 단어들을 mapping 되는 정수로 인코딩한다.

In [65]:
encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


### 2) Counter 사용하기

In [67]:
from collections import Counter

In [68]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


### 단어 집합 (vocabulary)를 만들기 위해 sentences에서 문장의 경계인 "[,]"를 제거하고 단어들을 하나의 리스트로 생성

In [70]:
words = sum(sentences, [])
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


### Counter()를 이용하여 중복을 제거하고 빈도수 기록

In [71]:
vocab = Counter(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


### most_common()는 상위 빈도수를 가진 주어진 수의 단어만을 리턴한다. 이를 사용하여 빈도수가 높은 단어들을 원하는 개수만큼만 얻을 수 있다.

In [72]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


### 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여한다.

In [74]:
word_to_index = {}
i = 0
for (word, frequency) in vocab:
    i += 1
    word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


## 3) NLTK의 FreqDist 사용하기

NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원한다. Counter()와 같은 방법으로 사용할 수 있다.

In [75]:
from nltk import FreqDist
import numpy as np

In [76]:
vocab = FreqDist(np.hstack(sentences)) # np.hstack()으로 문자 구분을 제거하여 입력으로 사용
print(vocab)

<FreqDist with 13 samples and 36 outcomes>


In [77]:
print(vocab['barber'])

8


In [78]:
print(vocab['word'])

2


In [79]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [81]:
word_to_index = { word[0] : index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


In [82]:
test = ['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test):
    print("value : {}, index : {}".format(value, index))

value : a, index : 0
value : b, index : 1
value : c, index : 2
value : d, index : 3
value : e, index : 4


# 2. Keras가 제공하는 Text preprocessing

In [83]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [84]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [85]:
tokenizer = Tokenizer()

In [86]:
tokenizer.fit_on_texts(sentences) # fit_on_texts()에 corpus를 입력으로 하면 빈도수를 기반으로 단어 집합을 생성하는 메소드

In [87]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [88]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [90]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [91]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [92]:
vocab_size = 5
words_frequency = [w for w, c in tokenizer.word_index.items() if c >= vocab_size + 1]
for w in words_frequency:
    del tokenizer.word_index[w]
    del tokenizer.word_counts[w]
print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


In [93]:
tokenizer = Tokenizer(num_words = vocab_size + 1)
tokenizer.fit_on_texts(sentences)

In [94]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [95]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [96]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


## One-hot Encoding

One-hot Encoding : 단어 집합을 활용하여 단어를 표현하는 가장 기본적인 방법

One-hot Encoding은 단어 집합의 크기를 vector(1차원 배열)의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에 0을 부여하는 vector 표현 방법

In [97]:
from konlpy.tag import Okt

In [98]:
okt = Okt()

In [99]:
token = okt.morphs("나는 자연어 처리를 배운다.")

In [100]:
token

['나', '는', '자연어', '처리', '를', '배운다', '.']

### 1. 정수 인코딩

In [101]:
word2index = {}
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5, '.': 6}


## 2. One-hot Encoding

In [102]:
def one_hot_encoding(word, word2index):
    one_hot_vector = [0] * (len(word2index))
    index = word2index[word]
    one_hot_vector[index] = 1
    
    return one_hot_vector

In [103]:
one_hot_encoding("자연어", word2index)

[0, 0, 1, 0, 0, 0, 0]

In [104]:
one_hot_encoding("배운다", word2index)

[0, 0, 0, 0, 0, 1, 0]

## 2. Keras를 이용한 One-hot encoding

In [105]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [106]:
text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

### (1) 정수 인코딩 

In [107]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
print(tokenizer.word_index)

{'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


In [108]:
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"

In [110]:
encoded = tokenizer.texts_to_sequences([sub_text])[0]
print(encoded)

[2, 5, 1, 6, 3, 7]


### (2) One-hot Encoding

In [111]:
one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


## One-hot Encoding 한계점

1. 단어 사전에 인덱스 부여된 단어 수에 따라 One-hot vector 크기가 결정된다. 메모리 낭비 요인이 있다.

2. 유사 단어에 대한 처리가 불가능

3. One-hot Encoding 한계점을 극복하기 위해 사용하는 방법. 카운트 기반 벡터 방식. 예측 기반 벡터 방식

# BPE(Byte Pair Encoding)

OOV(Out-of-Vocabulary) : 단어 집합에 포함되지 않은 단어, UNK(Unknown)